# Scaling training data

We have our basic workflow for label acquisition

1. Extract tokens
2. Show token in context and label token with sentiment (Repeat ad nauseum)
3. Run algorithm

But you may very well cringe at the *repeat ad nausem* part -- maybe you need the results next week for a big conference submission or maybe you just don't have the time to do the labelling task at the scale needed.  

Fortunately there are readily available options if this is the case. 

# Crowdsourcing

In theory, the concept is simple - get other people to tell you if a word is positive or negative. The difficult part is figuring out how to access such an audience that will also accept the remuneration you are able to provide.

Crowdsourcing is generally viewed as the answer to constructing a `training` dataset such as this. There a number of such platforms, but the oldest/most oft used is still Amazon's Mechanical Turk. Given that we will work through how to use this service programmatically

# The process of getting set up

Having an account with Amazon where you can buy shampoo and beef jerky is insufficient to use Mechanical Turk. MT resides (more or less) as a part of its enterprise services (which are labelled AWS-Amazon Web Services). To be able to be a requester on MT you must:

1. Sign up for an AWS account (aws.amazon.com)
2. Sign up for an MTurk requester account (requester.mturk.com)
3. Link Your MTurk account to your aws account (https://requester.mturk.com/developer)
4. Sign up for MTurk Sandbox, which is where you can test your forms without paying actual people (requestersandbox.mturk.com) an dlink your sandbox to the aws account (requestersandbox.mturk.com/developer)
5. Set up the IAM (Identity and Access Management) User

# Installation and basic AWS Access

Fortunately, there is a python package to manage access to AWS (boto3). First you will need to install this package with pip

In [ ]:
!pip install boto3

Then you will need to configure a credentials file that goes in your user directory (`~/.aws/credentials`) with your IAM account credentials. The `credentials` file should be structured as:

<pre>
[default]
aws_access_key_id = YOUR_KEY
aws_secret_access_key = YOUR_SECRET
</pre>

Then set up a configuration file to tell Amazon which region you want your services to be started in (`~/.aws/config`)?

<pre>
[default]
region=us-east-1
</pre>

And then you can check your sandbox balance

In [ ]:
import boto3

MTURK_SANDBOX = 'https://mturk-requester-sandbox.us-east-1.amazonaws.com'

mturk = boto3.client('mturk', endpoint_url = MTURK_SANDBOX)

In [ ]:
#Should have 10,000 available
print("Available sandbox balance: {0}".format(mturk.get_account_balance()['AvailableBalance']))

If you instead want to connect to your actual MTurk account and marketplace, you can just leave out the endpoint url

In [ ]:
real_mturk = boto3.client('mturk')
print("My real money: {0}".format(real_mturk.get_account_balance()['AvailableBalance']))

But I don't want to actually pay money yet

In [ ]:
del real_mturk

# Terminology

**Worker**: Anyone on the other side of the MTurk marketplace. Workers can view all open assignments and choose which ones to work on.

**HIT**: Human Intelligence Task - the single unit of work that a Turker would accept. This HIT could be a single task (i.e. "What is in this image") or a series of tasks (although that will increase length of time to complete and pay should scale with that factor). For the sake of further discussion we will say that labelling 1 word is 1 HIT and you have 100 words you want to label.

**Assignment**: Number of workers that should complete each HIT. If you set Assignment to 2 for 100 word HITs, then you would have 200 assignments. You will want to have an assignment of 3 or more when labelling words to increase confidence in the assigned score.

# Hit coding

Hits are ::drumroll please::.....HTML templates :( (Technically it is a HTML page that will be wrapped inside XML, so that's why we save it as `xml`)

That's right, you'll need to create a HTML page for your HIT that will be submitted. In its most basic form, it is relatively simple.

To make life easier, I separate this into 3 parts: `turk_hit_frontmatter.xml`, `turk_question.html`,`turk_hit_backmatter.xml`. The reason is that you can open the `html` page in a browser and see the result directly.

Then to make the final document to submit to AWS, it's just concatenating frontmatter, question, and backmatter to a new file (`backmatter` and `frontmatter` are constant). This front and back matter to the document is pretty simple too.

In [ ]:
print(open('turk_hit_frontmatter.xml').read())
print('----00000 Not in File 00000-------')
print(open('turk_hit_backmatter.xml').read())

And I've coded the simplest turk question possible to pair it with.

In [ ]:
!open turk_question.html

Constructing the final, submittable question is then relatively simple - it's just putting the three files together into one.

In [ ]:
def construct_turk_xml(turk_html):
    fulltext = open('turk_hit_frontmatter.xml').read() + open(turk_html).read() + \
               open('turk_hit_backmatter.xml').read()
    return fulltext
        
fulltext = construct_turk_xml('turk_question.html')

Now that the task creation is done, we can move to submitting the task.

In [ ]:
new_hit = mturk.create_hit(
    Title = 'Is the following word positive, neutral, or negative in emotion?',
    Description = 'Read the passage and click the button for the emotion that is attached to the bolded word',
    Keywords = 'text, quick, labeling',
    Reward = '0.01',
    MaxAssignments = 1,
    LifetimeInSeconds = 172800,
    AssignmentDurationInSeconds = 600,
    AutoApprovalDelayInSeconds = 10,
    Question = fulltext,
)

print( "https://workersandbox.mturk.com/mturk/preview?groupId=" + new_hit['HIT']['HITGroupId'] )
print( "HITID = " + new_hit['HIT']['HITId'] + " (Use to Get Results)" )

The fields mostly speak for themselves at the start. 

The reward is how much you will pay in USD cents (so this task is for 15 cents).

MaxAssignments is the number of turkers you want to complete the HIT

LifetimeInSeconds - how long the HIT should be available on the MTurk marketplace

AssignmentDurationInSeconds - how long the turker has to complete the HIT once they start the task

AutoApprovalDelayInSeconds - You have the ability to manually approve/deny a turker's work (which determines if the worker gets paid). This threshold sets when the system will move from manual to automatic approval (so that if you forget, the turker still gets paid). Note - requesters are rated on a separate forum for turkers and promptness of paying is one attribute that they track. Don't forget about paying in a reasonable amount of time, especially for low cost/risk tasks.

Question - what you want them to answer.

You can go check out the HIT at the sandbox link (need to register as a turker)

**Excellent**

Now (that I have most likely completed my own HIT), we should now be able to pull the data.

All will we will need is the client connection to MTurk and the HITID for our task.

In [ ]:
worker_results = mturk.list_assignments_for_hit(HITId=new_hit['HIT']['HITId'])
worker_results

And my answer is inside the assignments list:

In [ ]:
worker_results['Assignments'][0]

And now we see something ugly - the answer is in the `Answer` field, but it's in XML! 

Fortunately we can just install the `xmltodict` package which will convert the data out of xml and into something that's friendlier for our purposes.

In [ ]:
!pip install xmltodict

In [ ]:
import xmltodict

xml_doc = xmltodict.parse(worker_results['Assignments'][0]['Answer'])
xml_doc['QuestionFormAnswers']['Answer']

And there we go! We have our answer - the turker thinks that **unkindly** is *negative*

I will leave as an exercise for the reader to figure out how to automatically fill the html template with the passage and word of interest (Hint: manipulate it as a string in python)